In [2]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import random
import pandas as pd

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [4]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [5]:
import pandas as pd
import numpy as np
a= pd.read_parquet('./dataset.parquet')
a['track_id']= a['track_id'].astype('str')
a['string']=a['track_nm']+' '+a['artist_nm_list']
a= a.set_index(np.arange(len(a)))
b = a.sample(frac=1).reset_index(drop=True)
original = a['string'].values
similar = b['string'].values
new_df=pd.DataFrame({'sentence1':original, 'sentence2':similar,'label':0})

In [6]:
embeddings_1 = model.encode(new_df['sentence1'], convert_to_tensor=True)
embeddings_2 = model.encode(new_df['sentence2'], convert_to_tensor=True)
cosine_scores = util.pytorch_cos_sim(embeddings_1, embeddings_2)

OutOfMemoryError: CUDA out of memory. Tried to allocate 359.03 GiB (GPU 0; 79.10 GiB total capacity; 2.88 GiB already allocated; 3.53 GiB free; 2.91 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens")

tokenizer.tokenize(
  doc_list[0],
  return_tensors='pt',
  truncation=True,
  max_length=256,
  pad_to_max_length=True
)[0:20]

#Tokenize sentences
encoded_input = tokenizer(doc_list, padding=True, truncation=True, max_length=122, return_tensors='pt')

model = AutoModel.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens").to("cuda")

with torch.no_grad():
    model_output = model(input_ids=encoded_input["input_ids"].to("cuda"))

#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'].to("cuda"))

cosine_scores = util.pytorch_cos_sim(sentence_embeddings, sentence_embeddings)

In [ ]:
corpus = kor_list
corpus_embedding = model.encode(corpus, convert_to_tensor = True)
query = random.sample(kor_list,1)
query_embedding = model.encode(query, convert_to_tensor = True)
cos_score = util.pytorch_cos_sim(query_embedding, corpus_embedding)[0]
cos_score = cos_score.cpu()
top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]